In [1]:
import brightway2 as bw
import os               # to use "operating system dependent functionality"
import numpy as np      # "the fundamental package for scientific computing with Python"
import pandas as pd     # "high-performance, easy-to-use data structures and data analysis tools" for Python
import csv
import stats_arrays
import scipy as sp
import pandas as pd
import pickle

In [2]:
bw.projects.set_current('4 param distrib')

In [3]:
bw.bw2setup()

Creating default biosphere

Applying strategy: normalize_units

Writing activities to SQLite3 database:
0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:00



Applying strategy: drop_unspecified_subcategories
Applied 2 strategies in 0.01 seconds
Title: Writing activities to SQLite3 database:
  Started: 08/17/2017 11:49:00
  Finished: 08/17/2017 11:49:01
  Total time elapsed: 00:00:00
  CPU %: 70.200000
  Memory %: 1.280258
Created database: biosphere3
Creating default LCIA methods

Applying strategy: normalize_units
Applying strategy: set_biosphere_type
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Applied 4 strategies in 1.26 seconds
Wrote 718 LCIA methods with 178008 characterization factors
Creating core data migrations



# Creating new distribution: 4 parameter beta distribution

In [3]:
from numpy import *
from stats_arrays import *


class FourParamBetaUncertainty(UncertaintyBase):

    """
4 PARAM BETA DISTRIBUTIONS
The 4 parameters Beta distribution has the probability distribution function:

The :math:`\\alpha` parameter is ``loc``, and :math:`\\beta` is ``shape``. 
By default, the Beta distribution is defined from 0 to 1; the upper bound can be rescaled with the ``maximum`` parameter
and the lower bound with the ``minimum`` parameter.

Wikipedia: `Beta distribution <http://en.wikipedia.org/wiki/Beta_distribution>`_
    """
    id = 14
    description = "Four Parameters Beta uncertainty"

    @classmethod
    def validate(cls, params):
        scale_param=params['maximum']-params['minimum']
        if (params['loc'] > 0).sum() != params.shape[0]:
            raise InvalidParamsError("Real, positive alpha values are" +
                                     " required for Beta uncertainties.")
        if (params['shape'] > 0).sum() != params.shape[0]:
            raise InvalidParamsError("Real, positive beta values are" +
                                     " required for Beta uncertainties.")
        if (scale_param <= 0).sum():
            raise InvalidParamsError("Scale value must be positive or NaN")

    @classmethod
    def random_variables(cls, params, size, seeded_random=None,
                         transform=False):
        scale_param=params['maximum']-params['minimum']
        if not seeded_random:
            seeded_random = random
        scale = scale_param
        scale[isnan(scale)] = 1
        return scale.reshape((-1, 1)) * seeded_random.beta(params['loc'],params['shape'],size=(size,params.shape[0])).T + params['minimum']

    @classmethod
    def cdf(cls, params, vector):
        vector = cls.check_2d_inputs(params, vector)
        results = zeros(vector.shape)
        scale_param=params['maximum']-params['minimum']
        scale = scale_param
        scale[isnan(scale)] = 1
        for row in range(params.shape[0]):
            results[row, :] = stats.beta.cdf(vector[row, :],
                                             params['loc'][row], params['shape'][row],
                                             loc=params['minimum'][row],
                                             scale=scale[row])
        return results

    @classmethod
    def ppf(cls, params, percentages):
        percentages = cls.check_2d_inputs(params, percentages)
        results = zeros(percentages.shape)
        scale_param=params['maximum']-params['minimum']
        scale = scale_param
        scale[isnan(scale)] = 1
        for row in range(percentages.shape[0]):
            results[row, :] = stats.beta.ppf(percentages[row, :],
                                             params['loc'][row], params['shape'][row],
                                             loc=params['minimum'][row],
                                             scale=scale[row])
        return results

    @classmethod
    def statistics(cls, params):
        alpha = float(params['loc'])
        beta = float(params['shape'])
        mini = float(params['minimum'])
        maxi = float(params['maximum'])
        # scale = 1 if isnan(params['maximum'])[0] else float(params['maximum'])
        if alpha <= 1 or beta <= 1:
            mode = "Undefined"
        else:
            mode = mini + (maxi-mini) * (alpha - 1) / (alpha + beta - 2)
        return {
            'mean': mini + (maxi-mini) * alpha / (alpha + beta),
            'mode': mode,
            'median': "Not Implemented",
            'lower': mini,
            'upper': maxi
        }

    @classmethod
    def pdf(cls, params, xs=None):
        scale_param=params['maximum']-params['minimum']
        scale = 1 if isnan(scale_param)[0] else float(scale_param)
        if xs is None:
            xs = arange(0, scale, scale / cls.default_number_points_in_pdf)
        ys = stats.beta.pdf(xs, params['loc'], params['shape'],
                            loc=params['minimum'],
                            scale=scale)
        return xs, ys.reshape(ys.shape[1])

In [4]:
uncertainty_choices.add(FourParamBetaUncertainty)
print (FourParamBetaUncertainty in uncertainty_choices)

True


# Importing impact method with uncertain CF using 4 param beta distribution

In [6]:
input_filepath='C:\\bw2-python\\Notebook - tutorial\\IW+ integration\\Pickle Python object saved\\methods_with_errors_ID14.55a38b3752c9c63824403cd0633fdcc8.bw2package'
ic_to_test=bw.BW2Package.import_file(input_filepath)

In [8]:
ic_names_to_test=[ic_name for ic_name in bw.methods if 'with uncert ID14' in str(ic_name) 
                  and '- only with spatial variability - Four param beta integration' not in str(ic_name)]
ic_names_to_test

[('IMPACTWorld+ - Endpoint - update august 15th 2017',
  'Ecosystem Quality',
  'Land occupation, biodiversity, GLO, with uncert ID14'),
 ('IMPACTWorld+ - Endpoint - update august 15th 2017',
  'Ecosystem Quality',
  'Freshwater eutrophication, GLO, with uncert ID14'),
 ('IMPACTWorld+ - Endpoint - update august 15th 2017',
  'Ecosystem Quality',
  'Land transformation, biodiversity, GLO, with uncert ID14')]

# Importing database containing one activity with all elementary flows in BW2

In [16]:
input_filepath='C:\\bw2-python\\Notebook - tutorial\\IW+ integration\\Pickle Python object saved\\DB_For-test_v2.9536154a05725c74b8157d8b086f1aef.bw2package'
DB_test_list=bw.BW2Package.import_file(input_filepath)

Writing activities to SQLite3 database:
0%  100%
[#] | ETA: 00:00:00
Total time elapsed: 00:00:01


Title: Writing activities to SQLite3 database:
  Started: 08/18/2017 11:36:17
  Finished: 08/18/2017 11:36:18
  Total time elapsed: 00:00:01
  CPU %: 30.300000
  Memory %: 1.389069


# Monte Carlo calculation

In [19]:
#list_methods_name = IW_with_uncert
#fu_tuple = (act,1)
#iterations = 10

def MC_multi_impact(fu_tuple,list_methods_name,iterations):
    
    MC_results_dict={}
    fu_dict={fu_tuple[0]:fu_tuple[1]}
    
    for ic in list_methods_name:
        print(ic)
        MC_results_per_IC = bw.MonteCarloLCA(fu_dict,ic)
        scores = [next(MC_results_per_IC) for _ in range(iterations)]
        MC_results_dict[ic]=scores
    
    return MC_results_dict;

In [20]:
DB_test=bw.Database("For test_v2")
act=DB_test.get("All elementary flows")

list_methods_name = ic_names_to_test
fu_tuple = (act,1)
iterations = 10

scores=MC_multi_impact(fu_tuple,list_methods_name,iterations)
scores

('IMPACTWorld+ - Endpoint - update august 15th 2017', 'Ecosystem Quality', 'Land occupation, biodiversity, GLO, with uncert ID14')


MaximumIterationsError: 